In [ ]:
import itertools
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.feature_selection import mutual_info_classif as MIF
import random
import os
# chain()可以把一组迭代对象串联起来，形成一个更大的迭代器
from itertools import chain
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers, layers, losses, metrics
from tensorflow.keras.initializers import glorot_normal, he_normal
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Embedding, Lambda, multiply, Flatten, Concatenate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2

from deepctr.inputs import get_dense_input, create_embedding_matrix, embedding_lookup, get_dense_input, varlen_embedding_lookup, \
    get_varlen_pooling_list, mergeDict
from deepctr.layers.sequence import SequencePoolingLayer, Transformer, AttentionSequencePoolingLayer

from deepctr.feature_column import  SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names, build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM, FEFMLayer, BiInteractionPooling, AFMLayer, CIN, InteractingLayer, FwFMLayer, InnerProductLayer, OutterProductLayer, FGCNNLayer, CrossNet,  CrossNetMix
from deepctr.layers.utils import concat_func, add_func, Hash, NoMask, combined_dnn_input, reduce_sum, softmax
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Zeros, glorot_normal, RandomNormal
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def MNFMA(linear_feature_columns, dnn_feature_columns, fm_group=[DEFAULT_GROUP_NAME], dnn_hidden_units=(128, 128),
      l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, 
      seed=0, dnn_dropout=0,dropout = 0.1, bi_dropout=0, dnn_activation='relu', dnn_use_bn=False, task='binary'):
            # deepFM处理过程
            features = build_input_features(linear_feature_columns + dnn_feature_columns)

            inputs_list = list(features.values())

            linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear, )
            group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding,
                                                                        seed = seed, support_group=True)
            
#             dnn_input = combined_dnn_input(list(chain.from_iterable(group_embedding_dict.values())), dense_value_list)
            sparse_embedding_list = list(chain.from_iterable(group_embedding_dict.values()))
            fm_input = concat_func(sparse_embedding_list, axis=1)
            bi_out = BiInteractionPooling()(fm_input)
            if bi_dropout:
                bi_out = tf.keras.layers.Dropout(bi_dropout)(bi_out, training=None)
            dnn_input = combined_dnn_input([bi_out], dense_value_list)
    
            fm_logit = add_func([FM()(concat_func(v, axis=1))
                         for k, v in group_embedding_dict.items() if k in fm_group])
            dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                     dnn_use_bn, seed = seed)(dnn_input)
            dnn_output=tf.keras.layers.Dropout(dropout, seed = seed)(dnn_output)

            dnn_logit = tf.keras.layers.Dense(
                    1, use_bias=False, activation=None, kernel_initializer=glorot_normal(seed=seed))(dnn_output)
            fm_logit = Dense(1, use_bias=False)(fm_logit)
            dnn_logit = Dense(1, use_bias=False)(dnn_logit)
            final_logit = add_func([fm_logit, dnn_logit])
            output = PredictionLayer(task)(final_logit)
            #tensorflow模型拟合
            model = Model(inputs=[features], outputs=[output])
            #multi_category_focal_loss2(alpha=0.35, gamma=2)
            model.compile(optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
                          loss=losses.BinaryCrossentropy(), 
                          metrics=['AUC', 'binary_accuracy', 'Precision', 'Recall'])
            return model

In [ ]:
metrics = []
lrs = [1]
for i in lrs:
    metric = []
    for j in range(1):
        model = MNFMA(linear_feature_columns, dnn_feature_columns, fm_group=[DEFAULT_GROUP_NAME], dnn_hidden_units=(128, 128),
      l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, 
      seed=0, dnn_dropout=0,dropout = 0.1, bi_dropout=0, dnn_activation='relu', dnn_use_bn=False, task='binary')
        input_train = deepfm_train
        model.fit(input_train,
                        label,
                        validation_split=0.2,
                        epochs=100,
                        batch_size=128,
                        shuffle = False,
                        verbose = 1, 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],)
        input_test = deepfm_test
        ans_mtx = model.predict(input_test, batch_size=100)
        loss, auc, acc, pre, rec = model.evaluate(input_test, label1)
        metric.append([loss, auc, acc, pre, rec])
    metric = np.array(metric)
    print(metric)
    metrics.append(metric)
print(metrics)

In [ ]:
print(metrics)
print(*zip(lrs, np.mean(metrics, axis = 1)))